In [98]:
import gym
import torch
from myrl.buffers import ReplayBuffer
from myrl.visualizer import showit
from myrl.environments import Envs
from myrl.policies import RandomPolicy

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from myrl.utils import get_batch_a, get_batch_obs

envname = 'BipedalWalker-v3'
envname = 'LunarLanderContinuous-v2'
# envname = 'Pendulum-v0'
env = gym.make(envname)
envs = Envs(envname, 1)

In [168]:
entacter = lambda obs: (model.cross_entropy(torch.tensor(obs).float(), 3, 60, 25, pmax=0.15), -1)
gradactor= lambda obs: (gradient_optimize(model, obs, 20, [-2, 2], grad_step=1), -1)
showit(env, gradactor, max_steps=200, time_delta=0)

196 /200197 /200198 /200

-337.3980293244292

In [12]:
repbuff = ReplayBuffer(1000000)

In [172]:
class ModelResidual(nn.Module):
    def __init__(self, net_arch):
        super().__init__()
        l = [nn.Linear(a, b) for a, b in zip(net_arch[:-1], net_arch[1:])]
        l2 = [nn.Linear(net_arch[-2], 1)]
        self.layers = nn.ModuleList(l+l2)

    def forward(self, oldobs, a):
        # h = torch.tensor(x, dtype=torch.float)
        h = torch.cat((oldobs, a), dim=1)
        for lay in self.layers[:-2]:
            h = F.relu(lay(h))
        obs = self.layers[-2](h) + oldobs
        r = self.layers[-1](h)
        return r, obs

    def random_shooting(self, obs, nsmpls=100, tlen=10, all=False):
        maxr = float('-inf')
        obs = obs.repeat(nsmpls, 1)
        rs = torch.zeros(nsmpls, 1)
        for t in range(tlen):
            a = get_batch_a(env, nsmpls)
            r, obs = self.forward(obs, a)
            rs += r
            a = a.unsqueeze(1)
            if t==0:
                ass = a
            else:
                ass = torch.cat((ass, a), dim=1)
        # ii = torch.argmax(rs)
        if all:
            return ass, rs
        else:
            ii = torch.argmax(rs)
            return ass[ii][0].unsqueeze(-1)

    def cross_entropy(self, obs, niter, nsmpls, tlen, pmax=0.1, all=False):
        k = int(nsmpls*pmax)
        aret = None
        # print(niter)
        ass, rs = self.random_shooting(obs, nsmpls, tlen, all=True)
        obs = obs.repeat(nsmpls, 1)
        for iter in range(niter):
            topi = rs.squeeze(-1).topk(k)[1]
            topass = ass[topi]
            m, std = topass.mean(dim=0), topass.std(dim=0)
            dis = torch.distributions.Normal(m, std)
            ass = dis.sample(torch.Size((nsmpls,)))
            rs = self._eval_actions(obs, ass)
        if all:
            return ass, rs
        else:
            ii = torch.argmax(rs)
            return ass[ii][0].unsqueeze(-1) 

    def gradient_optimize(self, obs, tlen, minimax, ngrad_steps=10, grad_step=1e-1, nsmpls=1, all=False):
        obs = torch.tensor(obs).float()
        actions = torch.randn(nsmpls, tlen, 1, requires_grad=True)
        obs = obs.repeat(nsmpls, 1)
        opt = torch.optim.Adam([actions], lr=grad_step)
        for grad_step in range(ngrad_steps):
            rs = 0
            lobs = obs
            for i in range(tlen):
                r, lobs = self.forward(lobs, actions[:, i])
                rs += r
            loss = -rs.mean()
            opt.zero_grad()
            loss.backward()
            opt.step()
            with torch.no_grad():
                actions = torch.clamp(actions, -2, 2)
            
        ii = rs.squeeze(-1).argmax()
        if all:
            return actions[ii], rs[ii], actions
        return actions[ii][0].unsqueeze(0).detach().numpy()

    def _eval_actions(self, obs, ass):
        rs = 0
        for i in range(ass.shape[1]):
            a = ass[:, i]
            r, obs = self.forward(obs, a)
            rs += r
        return rs
            
    
adim = env.action_space.shape[0]
sdim = env.observation_space.shape[0]
idim = adim + sdim
odim = rdim + sdim
# model = ModelResidual([idim, 128, 64, sdim])
random_policy = RandomPolicy(env).act
obstest = get_batch_obs(env, 1)
model.cross_entropy(obstest, 10, 10, 10, pmax=0.5), adim, sdim

IndentationError: expected an indented block (<ipython-input-172-27a09dc96965>, line 65)

In [171]:
def gradient_optimize(self, obs, tlen, minimax, ngrad_steps=10, grad_step=1e-1, nsmpls=1, all=False):
    obs = torch.tensor(obs).float()
    actions = torch.randn(nsmpls, tlen, 1, requires_grad=True)
    obs = obs.repeat(nsmpls, 1)
    opt = torch.optim.Adam([actions], lr=grad_step)
    for grad_step in range(ngrad_steps):
        rs = 0
        lobs = obs
        for i in range(tlen):
            r, lobs = self.forward(lobs, actions[:, i])
            rs += r
        loss = -rs.mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        with torch.no_grad():
            actions = torch.clamp(actions, -2, 2)
        
    ii = rs.squeeze(-1).argmax()
    if all:
        return actions[ii], rs[ii], actions
    return actions[ii][0].unsqueeze(0).detach().numpy()
    
gradient_optimize(model, obstest, 10, [-2, 2], ngrad_steps=10, nsmpls=10, grad_step=1)



<ipython-input-171-e7a1e9298b54>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs = torch.tensor(obs).float()


array([[-0.2027486]], dtype=float32)

In [96]:
opt = torch.optim.Adam(model.parameters(), lr=5e-2)
model

ModelResidual(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=3, bias=True)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [97]:
# exname = 'model_bip_lander_big_residual'
# os.system('mkdir tb/' + exname)
# writer = SummaryWriter('tb/'+exname)

print(model)
bsize = 128
loss = 0; rloss=0; oloss=0;
for ep in range(10000):
    obs = env.reset()
    if ep%1==0:
        for step in range(1):
            oldobs, a, r, obs, d, _, _, _ = envs.rollout(random_policy)
            repbuff.add(oldobs, a, r, obs, d)
    # print("hahah")
    if bsize*3 > len(repbuff):
        print(len(rbuff), end='\r')
        continue
    oldobs, a, r, obs, d = repbuff.get(bsize)

    for opt_step in range(3):
        pr, pobs = model(oldobs, a)
        rloss = ((pr-r)**2).mean()
        oloss = ((pobs-obs)**2).mean()
        loss = rloss + 10*torch.abs(rloss/oloss).detach()*oloss
        opt.zero_grad()
        loss.backward()
        opt.step()
    # writer.add_scalar('loss/model', loss.item(), ep)
    # writer.add_scalar('loss/reward', rloss.item(), ep)
    # writer.add_scalar('loss/states', oloss.item(), ep)
    
    if ep%10==0:
        print(ep, loss.item(),rloss.item(), oloss.item())

ModelResidual(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=3, bias=True)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)
0 736.4976196289062 66.9543228149414 2.932508707046509
10 191.44430541992188 17.404027938842773 0.0034154325257986784
20 232.0092315673828 21.091747283935547 0.0022164222318679094
30 170.759033203125 15.52354907989502 0.0008411421440541744
40 124.36266326904297 11.305696487426758 0.0007627865415997803
50 135.18362426757812 12.289420127868652 0.0009586552623659372
60 111.11922454833984 10.1017484664917 0.001255122828297317
70 84.53765106201172 7.685240745544434 0.002242667367681861
80 75.86711883544922 6.897010326385498 0.0010846549412235618
90 106.01215362548828 9.637468338012695 0.000873371958732605
100 73.88268280029297 6.716607570648193 0.0006030031945556402
110 55.645835876464844 5.058712005615234 0.000

KeyboardInterrupt: 

In [32]:
model

Model(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=3, bias=True)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [64]:
def policy(obs, nsmpls=100, tlen=10):
    maxr = float('-inf')
    obs = obs.repeat(nsmpls, 1)
    rs = torch.zeros(nsmpls, 1)
    for t in range(tlen):
        a = nsample(env.action_space.sample, nsmpls)
#         print(a, "ass")
#         print(obs.shape, a.shape)
        r, obs = model(obs, a)
        rs += r
        a = a.unsqueeze(1)
        if t==0:
            ass = a
        else:
            ass = torch.cat((ass, a), dim=1)
    ii = torch.argmax(rs)
    #print(rs[ii])
#     print(ass)
    return ass[ii]

def policy2(obs, nsmpls=100, tlen=10):
    maxr = float('-inf')
    obs = obs.repeat(nsmpls, 1)
    rs = torch.zeros(nsmpls, 1)
    for t in range(tlen):
        a = get_batch_a(env, nsmpls)
        r, obs = model(obs, a)
        rs += r
        a = a.unsqueeze(1)
        if t==0:
            ass = a
        else:
            ass = torch.cat((ass, a), dim=1)
    ii = torch.argmax(rs)
    #print(rs[ii])
#     print(ass)
    return ass[ii]
   
def cross_entropy(obs, niter, nsmpls, tlen, pmax=0.1):
    k = int(nsmpls*pmax)
    



In [65]:
policy2(torch.zeros(1, 3))

tensor([[-0.2931],
        [-0.1762],
        [ 0.3904],
        [-0.2076],
        [ 1.2875],
        [-0.3306],
        [ 0.5876],
        [ 0.6570],
        [-0.0667],
        [ 0.0494]])

In [28]:
env.action_space.sample().shape, env.observation_space.sample().shape

((1,), (3,))

In [29]:
def nsample(smpl_func, n=100):
    toreturn = np.expand_dims(smpl_func(), axis=0)
    for i in range(n-1):
        new = np.expand_dims(smpl_func(), axis=0)
        toreturn = np.concatenate((toreturn, new), axis=0)
    return torch.tensor(toreturn, dtype=torch.float)

nsample(env.action_space.sample).shape


NameError: name 'np' is not defined

In [8]:
def showit():
    dons = 0
    observation = env.reset()
    for t in range(2000):
            env.render()
            obser = np.expand_dims(observation, axis=0)
            ass = policy(torch.tensor(obser).float(), 100, 30)
            observation, reward, done, info = env.step(ass[0].numpy())
            #observation, reward, done, info = env.step(env.action_space.sample())
            rp, op = model(torch.tensor(obser).float(), ass[0].unsqueeze(0))
            print(rp.item())
            print(reward)
            print(op)
            print(obs)
            print(("---"+str(t)+"---")*10)
            
            dons += done
            if done:
                print("r", reward)
                break
            time.sleep(0.005)
            print(t, "/200", reward, end='\r')

env.close()
cv2.destroyAllWindows()

In [10]:
showit()

AddBackward0>)
tensor([[ 0.9948,  0.1020,  1.7881],
        [-0.8947, -0.4468,  7.4100],
        [-0.8634,  0.5046, -5.6968],
        ...,
        [ 0.0617, -0.9981,  5.1603],
        [ 0.4373,  0.8993, -1.4300],
        [ 0.4022, -0.9155, -1.6348]])
---133------133------133------133------133------133------133------133------133------133---
133 /200 -0.009278881078197005-0.04819381237030029
-0.014081505706157266
tensor([[ 0.9962,  0.0425, -0.1520]], grad_fn=<AddBackward0>)
tensor([[ 0.9948,  0.1020,  1.7881],
        [-0.8947, -0.4468,  7.4100],
        [-0.8634,  0.5046, -5.6968],
        ...,
        [ 0.0617, -0.9981,  5.1603],
        [ 0.4373,  0.8993, -1.4300],
        [ 0.4022, -0.9155, -1.6348]])
---134------134------134------134------134------134------134------134------134------134---
134 /200 -0.014081505706157266-0.02733445167541504
-0.004701965602051974
tensor([[ 0.9986,  0.0429, -0.0174]], grad_fn=<AddBackward0>)
tensor([[ 0.9948,  0.1020,  1.7881],
        [-0.8947, -0.446

KeyboardInterrupt: 

In [89]:
x = torch.arange(1., 6.)
x.topk(k=2)[0]
x.argmax()

tensor(4)